Title: K-means clustering

Authors: Bauyrzhan Taimanov - Madina Amanatayeva

Date: Jan 18, 2024

In [8]:
# imports 
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from matplotlib import pyplot as plt

# 0) Context


## Customer Segmentation with Parallel K-Means

In business, we often say "Customer is king" and "Customer is always right." Basically, it means we need to focus on what customers want. So, the idea here is to use this concept to group products that match what different groups of customers like. This can help a lot in online stores because it means showing customers the products they're most likely to be interested in.

![alt text](https://editor.analyticsvidhya.com/uploads/73654cluster.jpg)

This concept can be handy in places where we use digital marketing tools to promote products. For example, in some types of ads, we show products to people based on things like their interests. Then, by looking at whether they click on the ads or not, we can figure out who else might like those products.
Similarly, in other types of ads, we bid for space to show our products to people in real-time. We do this based on what we know about the people seeing the ads and what's worked well in the past. This way, we can make sure we're showing the right products to the right people at the right time.

## Concept

Customers buy products based on different needs and constraints, like budget and preference for certain items. To understand these behaviors, we can analyze customer invoice data and other non-personal data.
Here's a simplified approach:
* Analyze customer invoice data to find buying patterns.
* Use clustering algorithms to group customers based on similar purchase behaviors.
* Identify products that match each customer group's preferences.
* Tailor digital campaigns to each customer group for better targeting.

Though, with our current data limitations, we may not have all the necessary details to build a complete model.

# 1) Analysis of the Serial Algorithm

## K-Means Algorithm 

K-means clustering stands as a well-known machine learning technique utilized for customer segmentation, driven by identifying similarities among them. Its goal is to discern k clusters within the data, with each cluster denoting a cohort of customers sharing common traits. The algorithm hinges on centroids, acting as the central points of these clusters, and employs distance measures to allocate customers into the relevant clusters.


![alt text](https://upload.wikimedia.org/wikipedia/commons/thumb/e/ea/K-means_convergence.gif/440px-K-means_convergence.gif)

## How does algorithm work?

Firstly, we have to import our data

In [9]:
df = pd.read_excel('Online Retail.xlsx')
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
